## Library Imports

In [1]:
import os
import warnings
import pickle
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import SubsetRandomSampler

import numpy as np
%matplotlib inline

import bts.dataset as dataset
import bts.model as model
import bts.classifier as classifier
import bts.plot as plot

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Computation Details')
print(f'\tDevice Used: ({device})\n')

print('Packages Used Versions:-')
print(f'\tPytorch Version: {torch.__version__}')

Computation Details
	Device Used: (cuda)  GeForce GTX 1050 Ti

Packages Used Versions:-
	Pytorch Version: 1.0.1


## Hyperparameters Tuning 

In [2]:
TEST_SPLIT = 0.2
BATCH_SIZE = 6
DATASET_USED = 'png_dataset'
DATASET_PATH = os.path.join('dataset',DATASET_USED)
EPOCHS = 100
FILTER_LIST = [16,32,64,128,256]
TRAIN = False
LOAD_MODEL = True
SAVE_MODEL = False
MODEL_NAME = f"UNet-{FILTER_LIST}.pt"

print(f"Model Name :   {MODEL_NAME}")

Model Name :   UNet-[16, 32, 64, 128, 256].pt


## Dataset Loading

In [10]:
def get_indices(length, new=False):

    
    file_path = os.path.join('dataset',f'split_indices_{DATASET_USED}.p')
    data = dict()
    if os.path.isfile(file_path) and not new:
        
        with open(file_path,'rb') as file :
            data = pickle.load(file)
            return data['train_indices'], data['test_indices']
    else:
        
        indices = list(range(length))
        np.random.shuffle(indices)
        split = int(np.floor(TEST_SPLIT * len(tumor_dataset)))
        train_indices , test_indices = indices[split:], indices[:split]
        
        data['train_indices'] = train_indices
        data['test_indices'] = test_indices
        with open(file_path,'wb') as file:
            pickle.dump(data,file)
    return train_indices, test_indices

In [4]:
import os
import bts.dataset as dataset

DATASET_PATH = 'Brain-Tumor-Segmentation-master\Brain-Tumor-Segmentation-master\dataset\png_dataset'

tumor_dataset = dataset.TumorDataset(DATASET_PATH)
tumor_dataset = dataset.TumorDataset(DATASET_PATH)

train_indices, test_indices = get_indices(len(tumor_dataset))
train_sampler, test_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(test_indices)

trainloader = torch.utils.data.DataLoader(tumor_dataset, BATCH_SIZE, sampler=train_sampler)
testloader = torch.utils.data.DataLoader(tumor_dataset, 1, sampler=test_sampler)

## Model Declaration

In [5]:
unet_model = None
unet_classifier = None

if not LOAD_MODEL:
  
    unet_model = model.DynamicUNet(FILTER_LIST).to(device)
    unet_classifier = classifier.BrainTumorClassifier(unet_model,device)
else:
    
    unet_model = model.DynamicUNet(FILTER_LIST)
    unet_classifier = classifier.BrainTumorClassifier(unet_model,device)
    
    
    state_dict = torch.load(os.path.join('saved_models', MODEL_NAME), map_location=torch.device('cpu'))
    unet_model.load_state_dict(state_dict)
    unet_model.to(device)  # Move the model to the desired device (CPU)

print('Saved model loaded')


Saved model loaded


## Model Training

In [2]:
class BrainTumorClassifier:
    def __init__(self, model, device):
        self.model = model
        self.device = device

    def test(self, testloader, threshold=0.5):
        self.model.eval()
        total_score = 0.0
        total_samples = 0

        for data in testloader:
            input_data, target_data = data['image'].to(self.device), data['mask'].to(self.device)
            output = self.model(input_data)
        
            dice_score = dice_coefficient(output, target_data)
            total_score += dice_score.item()
            total_samples += 1

        average_score = total_score / total_samples
        return average_score


## Training Results

### Training Loss Graph

In [ ]:
path = os.path.join('saved_models', MODEL_NAME) if SAVE_MODEL else None

unet_train_history = unet_classifier.train(EPOCHS, trainloader, mini_batch=100, save_best=path)
save_plot = os.path.join('images', f'{MODEL_NAME}-loss_graph.png')
plot.loss_graph(unet_train_history['train_loss'], save_plot)


### Visualize Test Dataset Predictions

In [ ]:
image_index = test_indices[i]
sample = tumor_dataset[image_index]
image, mask, output, d_score = unet_classifier.predict(sample,0.65)
title = f'Name: {image_index}.png   Dice Score: {d_score:.5f}'
# save_path = os.path.join('images',f'{d_score:.5f}_{image_index}.png')
plot.result(image,mask,output,title,save_path=None)
i += 1
if i >= len(test_indices):
    i = 0 